### Making the mapped.csv consistent

In [1]:
import pandas as pd
import pandas as pd
from matplotlib import pyplot
import statsmodels as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import statistics
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'
from statsmodels.tsa.arima_model import ARIMA
import itertools

In [2]:
df = pd.read_csv('mapped.csv')

In [3]:
def random_project(df):
    df = df.sample(n=1)
    project_number = df['Project Number'].item()
    return project_number

In [4]:
df.columns

Index(['Unnamed: 0', 'Fiscal Period Code', 'Project_Number', 'Project Id',
       'Project_Name', 'Health and Safety Status', 'project_date_create',
       'Project Status', 'SalesCloudClientNo',
       'Related Opportunity Project Number', 'Contract Type Name',
       'Ultimate Customer AUS Market Sector', 'Division Short Name',
       'location_2', 'location_4', 'Region', 'City', 'Country', 'Country Name',
       'Proj_CVP', 'Proj_Serv_Offering', 'CVP Description', '﻿CVP Code',
       'primary_client_industry', 'Sales Cloud Opportunity Number',
       'Funding Amount - ITC (Snapshot)', 'Backlog NR',
       'Actual Direct Labor Cost - MTD', 'Actual Non-Labor Cost - MTD',
       'Actual Gross Revenue - MTD', 'Actual Net Revenue - MTD', 'EBITA - MTD',
       'Forecast Direct Labor Cost - ITC', 'Forecast Net Revenue - ITC',
       'Order Intake (NR) - MTD', 'WBS1', 'bookings_jtd', 'interest_current',
       'interest_jtd', 'interest_ytd', 'multiplier_budget',
       'multiplier_current',

In [5]:
df = df.rename(columns={'Project_Number': 'Project Number', 'Project_Name' : 'Project Name'})
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Project Number'] = df['Project Number'].astype('str') 
#df['Fiscal Period Code'] = pd.to_datetime(df['Fiscal Period Code'], format='%Y%m') 

### Let's take out any project numbers that are not unique per time period

In [57]:
df1[['Project Number']] = df1[['Project Number']].apply(lambda x: x.str.lower())
df2 = df1.drop_duplicates(subset=['Fiscal Period Code','Project Number'], keep = 'first')
len(df2)

1100712

In [ ]:
random_project(df2)

### Let's add a column for the total project duration

In [76]:
df2['Projectduration'] = df2.groupby(['Project Number'])['Fiscal Period Code'].transform('count')

In [78]:
df2.columns

Index(['Fiscal Period Code', 'Project Number', 'Project Id', 'Project Name',
       'Health and Safety Status', 'project_date_create', 'Project Status',
       'SalesCloudClientNo', 'Related Opportunity Project Number',
       'Contract Type Name', 'Ultimate Customer AUS Market Sector',
       'Division Short Name', 'location_2', 'location_4', 'Region', 'City',
       'Country', 'Country Name', 'Proj_CVP', 'Proj_Serv_Offering',
       'CVP Description', '﻿CVP Code', 'primary_client_industry',
       'Sales Cloud Opportunity Number', 'Funding Amount - ITC (Snapshot)',
       'Backlog NR', 'Actual Direct Labor Cost - MTD',
       'Actual Non-Labor Cost - MTD', 'Actual Gross Revenue - MTD',
       'Actual Net Revenue - MTD', 'EBITA - MTD',
       'Forecast Direct Labor Cost - ITC', 'Forecast Net Revenue - ITC',
       'Order Intake (NR) - MTD', 'WBS1', 'bookings_jtd', 'interest_current',
       'interest_jtd', 'interest_ytd', 'multiplier_budget',
       'multiplier_current', 'multiplier_t

In [8]:
df3 = df2.drop_duplicates(subset=['Project Number'], keep = 'last')

def analysis(column):
    print(df3[column].unique())
    sub = df3.groupby(column).count()
    # plot the dataframe 
    return sub.plot(y=["Project Number"], kind="bar", figsize=(10,8))

In [77]:
len(df3), len(df2)

(63522, 1100712)

In [27]:
#analysis('Projectduration')

### Let's add a columm to add up all actual net revenue (cumulative sum)

In [58]:
df2['ActualNR_sum'] = df2.groupby("Project Number")['Actual Net Revenue - MTD'].cumsum()

### Let's see which SalescloudClientNO has most projects

In [85]:
df_combo=df2.groupby([ "SalesCloudClientNo", "Project Number"] ).size().to_frame(name = 'count').reset_index()
dfdf_combo=df_combo.groupby('SalesCloudClientNo').count()
dfdf_combo.pop('Project Number')


#This should be the names with more than two numbers
dfdf_combo.reset_index(inplace=True)
dfdf_combo.sort_values('count', 
                      inplace=True, 
                      ascending=False)
dfdf_combo

,SalesCloudClientNo,count
8,99999.0,15470
93,10238092.0,3111
993,10239479.0,2244
175,10238206.0,1396
980,10239458.0,975
...,...,...
680,10238990.0,1
1725,10240858.0,1
1726,10240859.0,1
1727,10240860.0,1


### Let's see which project types have most revenue

In [55]:
def average_nr(column):
    dff=df3.groupby([column])['ActualNR_sum'].sum().round().reset_index()
    dfff=df3.groupby([column])['Project Number'].count().reset_index()
    dff['ActualNR_sum'] = (dff['ActualNR_sum'] / dfff['Project Number']).round()
    
    fig, ax = plt.subplots(figsize=(10,8))
    bars = ax.bar(dff[column], dff['ActualNR_sum'], width=0.5)
    plt.xticks(rotation=90)
    for bar in bars:
        height = bar.get_height()
        label_x_pos = bar.get_x() + bar.get_width() / 2
        ax.text(label_x_pos, height, s=f'{height}', ha='center',va='bottom')

In [28]:
#average_nr('Division Short Name')

In [100]:
df3.groupby(['Division Short Name'])['ActualNR_sum'].sum()

Division Short Name
Enabling Function   -2.403110e+05
Environment          1.595409e+09
Infrastructure       4.978442e+08
Water                8.007870e+08
Name: ActualNR_sum, dtype: float64

In [29]:
#average_nr('Contract Type Name')

In [30]:
#average_nr('Proj_CVP')

In [31]:
#average_nr('Ultimate Customer AUS Market Sector')

In [86]:
df3.groupby(['Division Short Name'])['ActualNR_sum'].sum()

Index(['Fiscal Period Code', 'Project Number', 'Project Id', 'Project Name',
       'Health and Safety Status', 'project_date_create', 'Project Status',
       'SalesCloudClientNo', 'Related Opportunity Project Number',
       'Contract Type Name', 'Ultimate Customer AUS Market Sector',
       'Division Short Name', 'location_2', 'location_4', 'Region', 'City',
       'Country', 'Country Name', 'Proj_CVP', 'Proj_Serv_Offering',
       'CVP Description', '﻿CVP Code', 'primary_client_industry',
       'Sales Cloud Opportunity Number', 'Funding Amount - ITC (Snapshot)',
       'Backlog NR', 'Actual Direct Labor Cost - MTD',
       'Actual Non-Labor Cost - MTD', 'Actual Gross Revenue - MTD',
       'Actual Net Revenue - MTD', 'EBITA - MTD',
       'Forecast Direct Labor Cost - ITC', 'Forecast Net Revenue - ITC',
       'Order Intake (NR) - MTD', 'WBS1', 'bookings_jtd', 'interest_current',
       'interest_jtd', 'interest_ytd', 'multiplier_budget',
       'multiplier_current', 'multiplier_t

In [32]:
#average_nr('Risk Profile')

In [33]:
#analysis('Risk Profile')

In [34]:
#average_nr('Projectduration')

In [17]:
random_project(df2)

'30014265'

In [14]:
df2['Fiscal Period Code'] = pd.to_datetime(df2['Fiscal Period Code'], format='%Y%m') 

In [22]:
df_30012521 = df2[df2['Project Number'] == '30012521']
df_30014265 = df2[df2['Project Number'] == '30014265']
dfdf = pd.concat([df_30012521, df_30014265])

### Let's subset healthcare projects

In [25]:
dfdf = df2[df2['Risk Profile'] == 'Low']
dfdf.pivot(index="Fiscal Period Code", columns="Project Number", values="Actual Net Revenue - MTD").plot(figsize=(10,8))
#plt.text(1,1,'Pre', transform=ax3.transAxes)

### Let's make sure that some of the categorical variables are consistent throughout the dataset

We should keep Fiscal period code, Project Number, Project status, SalesCloudClientNo, contract type, Ultimate Customer AUS Market Sector, Division Short Name, Region, City, Proj_CVP, Proj_Serv_Offering, , primary_client_industry, Funding Amount - ITC (Snapshot), Backlog NR, Actual Direct Labor Cost - MTD, Actual Non-Labor Cost - MTD, Actual Gross Revenue - MTD, Actual Net Revenue - MTD, EBITA - MTD, Forecast Direct Labor Cost - ITC, Forecast Net Revenue - ITC, Order Intake (NR) - MTD,  Order Intake (NR) - MTD, Baseline NR, Break Even Multiplier - Project, Organization Size, Risk Profile, riskrating_project


(Optional) bookings_jtd	interest_current	interest_jtd	interest_ytd	multiplier_budget	multiplier_current	multiplier_target	overhead_current	overhead_rate, Total WIP


drag out from last (Ultimate Customer AUS Market Sector, Risk Profile


drag out from first (Proj_CVP, Proj_Serv_Offering, Region, City, primary_client_industry, riskrating_project, 

In [35]:
df_first = df2.drop_duplicates(subset=['Project Number'], keep = 'first')
df_last = df2.drop_duplicates(subset=['Project Number'], keep = 'last')

In [59]:
def cohesive(column, subdf):
    #subset df
    realData = pd.Series(data=subdf[column].values, index=subdf['Project Number'])
    #map back to the original dataset
    df2[column] = df2['Project Number'].map(realData).fillna(df2[column])
    #Use this to doublcheck
    return df2[df2['Project Number'] == '30013144'][column]

## Ultimate Customer AUS Market Sector (last)

In [60]:
cohesive('Ultimate Customer AUS Market Sector', df_last)

1275653    Chemicals
1335023    Chemicals
1359354    Chemicals
1396098    Chemicals
1423955    Chemicals
1469937    Chemicals
1493935    Chemicals
1535775    Chemicals
1556337    Chemicals
1592947    Chemicals
1616658    Chemicals
1638992    Chemicals
1669798    Chemicals
1707202    Chemicals
1713790    Chemicals
1739518    Chemicals
1790811    Chemicals
1800145    Chemicals
1836013    Chemicals
1861736    Chemicals
1895273    Chemicals
1952453    Chemicals
1970469    Chemicals
2005281    Chemicals
2039596    Chemicals
2071608    Chemicals
2102404    Chemicals
2177162    Chemicals
Name: Ultimate Customer AUS Market Sector, dtype: object

### Risk Profile (last)

In [61]:
cohesive('Risk Profile', df_last)

1275653    Low
1335023    Low
1359354    Low
1396098    Low
1423955    Low
1469937    Low
1493935    Low
1535775    Low
1556337    Low
1592947    Low
1616658    Low
1638992    Low
1669798    Low
1707202    Low
1713790    Low
1739518    Low
1790811    Low
1800145    Low
1836013    Low
1861736    Low
1895273    Low
1952453    Low
1970469    Low
2005281    Low
2039596    Low
2071608    Low
2102404    Low
2177162    Low
Name: Risk Profile, dtype: object

### drag out from first Proj_CVP, Proj_Serv_Offering, Region, City, primary_client_industry, riskrating_project,

In [62]:
cohesive('riskrating_project', df_first)

1275653    1.0
1335023    1.0
1359354    1.0
1396098    1.0
1423955    1.0
1469937    1.0
1493935    1.0
1535775    1.0
1556337    1.0
1592947    1.0
1616658    1.0
1638992    1.0
1669798    1.0
1707202    1.0
1713790    1.0
1739518    1.0
1790811    1.0
1800145    1.0
1836013    1.0
1861736    1.0
1895273    1.0
1952453    1.0
1970469    1.0
2005281    1.0
2039596    1.0
2071608    1.0
2102404    1.0
2177162    1.0
Name: riskrating_project, dtype: float64

In [63]:
cohesive('Proj_CVP', df_first)

1275653    Site Evaluation and Restoration
1335023    Site Evaluation and Restoration
1359354    Site Evaluation and Restoration
1396098    Site Evaluation and Restoration
1423955    Site Evaluation and Restoration
1469937    Site Evaluation and Restoration
1493935    Site Evaluation and Restoration
1535775    Site Evaluation and Restoration
1556337    Site Evaluation and Restoration
1592947    Site Evaluation and Restoration
1616658    Site Evaluation and Restoration
1638992    Site Evaluation and Restoration
1669798    Site Evaluation and Restoration
1707202    Site Evaluation and Restoration
1713790    Site Evaluation and Restoration
1739518    Site Evaluation and Restoration
1790811    Site Evaluation and Restoration
1800145    Site Evaluation and Restoration
1836013    Site Evaluation and Restoration
1861736    Site Evaluation and Restoration
1895273    Site Evaluation and Restoration
1952453    Site Evaluation and Restoration
1970469    Site Evaluation and Restoration
2005281    

In [64]:
cohesive('Proj_Serv_Offering', df_first)

1275653    EN - Environmental Planning
1335023    EN - Environmental Planning
1359354    EN - Environmental Planning
1396098    EN - Environmental Planning
1423955    EN - Environmental Planning
1469937    EN - Environmental Planning
1493935    EN - Environmental Planning
1535775    EN - Environmental Planning
1556337    EN - Environmental Planning
1592947    EN - Environmental Planning
1616658    EN - Environmental Planning
1638992    EN - Environmental Planning
1669798    EN - Environmental Planning
1707202    EN - Environmental Planning
1713790    EN - Environmental Planning
1739518    EN - Environmental Planning
1790811    EN - Environmental Planning
1800145    EN - Environmental Planning
1836013    EN - Environmental Planning
1861736    EN - Environmental Planning
1895273    EN - Environmental Planning
1952453    EN - Environmental Planning
1970469    EN - Environmental Planning
2005281    EN - Environmental Planning
2039596    EN - Environmental Planning
2071608    EN - Environme

In [65]:
cohesive('Region', df_first)

1275653    Illinois, US
1335023    Illinois, US
1359354    Illinois, US
1396098    Illinois, US
1423955    Illinois, US
1469937    Illinois, US
1493935    Illinois, US
1535775    Illinois, US
1556337    Illinois, US
1592947    Illinois, US
1616658    Illinois, US
1638992    Illinois, US
1669798    Illinois, US
1707202    Illinois, US
1713790    Illinois, US
1739518    Illinois, US
1790811    Illinois, US
1800145    Illinois, US
1836013    Illinois, US
1861736    Illinois, US
1895273    Illinois, US
1952453    Illinois, US
1970469    Illinois, US
2005281    Illinois, US
2039596    Illinois, US
2071608    Illinois, US
2102404    Illinois, US
2177162    Illinois, US
Name: Region, dtype: object

In [66]:
cohesive('City', df_first)

1275653    Kankakee
1335023    Kankakee
1359354    Kankakee
1396098    Kankakee
1423955    Kankakee
1469937    Kankakee
1493935    Kankakee
1535775    Kankakee
1556337    Kankakee
1592947    Kankakee
1616658    Kankakee
1638992    Kankakee
1669798    Kankakee
1707202    Kankakee
1713790    Kankakee
1739518    Kankakee
1790811    Kankakee
1800145    Kankakee
1836013    Kankakee
1861736    Kankakee
1895273    Kankakee
1952453    Kankakee
1970469    Kankakee
2005281    Kankakee
2039596    Kankakee
2071608    Kankakee
2102404    Kankakee
2177162    Kankakee
Name: City, dtype: object

In [67]:
cohesive('primary_client_industry', df_first)

1275653    CP - Chem/Pharma
1335023    CP - Chem/Pharma
1359354    CP - Chem/Pharma
1396098    CP - Chem/Pharma
1423955    CP - Chem/Pharma
1469937    CP - Chem/Pharma
1493935    CP - Chem/Pharma
1535775    CP - Chem/Pharma
1556337    CP - Chem/Pharma
1592947    CP - Chem/Pharma
1616658    CP - Chem/Pharma
1638992    CP - Chem/Pharma
1669798    CP - Chem/Pharma
1707202    CP - Chem/Pharma
1713790    CP - Chem/Pharma
1739518    CP - Chem/Pharma
1790811    CP - Chem/Pharma
1800145    CP - Chem/Pharma
1836013    CP - Chem/Pharma
1861736    CP - Chem/Pharma
1895273    CP - Chem/Pharma
1952453    CP - Chem/Pharma
1970469    CP - Chem/Pharma
2005281    CP - Chem/Pharma
2039596    CP - Chem/Pharma
2071608    CP - Chem/Pharma
2102404    CP - Chem/Pharma
2177162    CP - Chem/Pharma
Name: primary_client_industry, dtype: object

In [82]:
df2.to_csv('preprocessed.csv')

In [83]:
len(df2)

1100712